# Introduction to AI
## Text and Speech Demos


## Performing Basic Frequency Analysis
Let's start by using some very basic frequency analysis on a document to see if we can determine what the document is about  based on word frequency.

### Load a Text Document
Run the cell below to load the transcript of a Shark Tank pitch.

In [ ]:
# Open a Shark Tank pitch by Pick-up Pools
!wget -O SharkTank.txt https://raw.githubusercontent.com/markcubanai/Language/master/SharkTank.txt
doc1 = open("SharkTank.txt", "r")

# Read the document and print its contents
doc1Txt = doc1.read()
print(doc1Txt)

#### Normalize the Text

Text includes a lot of punctuation, which we need to remove if we want to work only with the actual words.

Run the cell below to strip all the punctuation from the text and convert the words to lower case.

In [ ]:
from string import punctuation

# remove numeric digits
txt = ''.join(c for c in doc1Txt if not c.isdigit())

# remove punctuation and make lower case
txt = ''.join(c for c in txt if c not in punctuation).lower()

# print the normalized text
print (txt)

#### Get the Frequency Distribution
Now let's tokenize the text (split it into individual words), and count the number of times each word occurs.

In [ ]:
import nltk
import pandas as pd
from nltk.probability import FreqDist
nltk.download("punkt")

# Tokenize the text into individual words
words = nltk.tokenize.word_tokenize(txt)

# Get the frequency distribution of the words into a data frame
fdist = FreqDist(words)
count_frame = pd.DataFrame(fdist, index =[0]).T
count_frame.columns = ['Count']
print (count_frame)

#### Visualize Word Frequency
It's often easier to analyze frequency by creating a visualization, such as a Pareto chart.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Sort the data frame by frequency
counts = count_frame.sort_values('Count', ascending = False)

# Display the top 60 words as a bar plot
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()    
counts['Count'][:60].plot(kind = 'bar', ax = ax)
ax.set_title('Frequency of the most common words')
ax.set_ylabel('Frequency of word')
ax.set_xlabel('Word')
plt.show()

#### Remove stop words

A large number of the words in the text are common words like "the" or "and". These "stopwords" add little in the way of semantic meaning to the text, and won't help us determine the subject matter - so run the cell below to remove them.

In [ ]:
# Get standard stop words from NLTK
nltk.download("stopwords")
from nltk.corpus import stopwords

# Filter out the stop words
txt = ' '.join([word for word in txt.split() if word not in (stopwords.words('english'))])

# Get the frequency distribution of the remaining words
words = nltk.tokenize.word_tokenize(txt)
fdist = FreqDist(words)
count_frame = pd.DataFrame(fdist, index =[0]).T
count_frame.columns = ['Count']

# Plot the frequency of the top 60 words
counts = count_frame.sort_values('Count', ascending = False)
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()    
counts['Count'][:60].plot(kind = 'bar', ax = ax)
ax.set_title('Frequency of the most common words')
ax.set_ylabel('Frequency of word')
ax.set_xlabel('Word')
plt.show()



### Visualize the Frequency Distribution for the Remaining Words
Now that we've prepared the text, we can tokenize the string into a list of individual words, and then perform frequency analysis on those words based on how often they appear in the text.

In [ ]:
# remind ourselves of the first document
print("------------------------------------------------")
print("Shark Tank Pick-up Pools Pitch")
print("------------------------------------------------")
print(doc1Txt)

# Load the movie script of the opening scene for Spider-Man: Into the Spider-Verse
!wget -O Spiderman.txt https://raw.githubusercontent.com/markcubanai/Language/master/Spiderman.txt
doc2 = open("Spiderman.txt", "r")
doc2Txt = doc2.read()
print("------------------------------------------------")
print("Spider-Man: Into the Spider-Verse")
print("------------------------------------------------")
print (doc2Txt)
from string import punctuation
txt2 = ''.join(c for c in doc2Txt if not c.isdigit())
txt2 = ''.join(c for c in txt2 if c not in punctuation).lower()
txt2 = ' '.join([word for word in txt2.split() if word not in (stopwords.words('english'))])


# Load the play-by-play transcript of the Dallas Mavericks' Luka Doncic hitting a three pointer at the buzzer to win Game 4 of the NBA Playoffs against the Clippers
!wget -O PlayByPlay.txt https://raw.githubusercontent.com/markcubanai/Language/master/PlayByPlay.txt
print("------------------------------------------------")
print("NBA Playoffs Luka Doncic Game Winner - Play by Play")
print("------------------------------------------------")
doc3 = open("PlayByPlay.txt", "r")
doc3Txt = doc3.read()
print (doc3Txt)
from string import punctuation
txt3 = ''.join(c for c in doc3Txt if not c.isdigit())
txt3 = ''.join(c for c in txt3 if c not in punctuation).lower()
txt3 = ' '.join([word for word in txt3.split() if word not in (stopwords.words('english'))])


#### Get TF-IDF Values for the top three words in each document

In the previous example, we've used basic term frequency to determine each word's "importance" based on how often it appears in the document. When dealing with a large corpus of multiple documents, a more commonly used technique is *term frequency, inverse document frequency* (or TF-IDF) in which a score is calculated based on how often a word or term appears in one document compared to its more general frequency across the entire collection of documents. Using this technique, a high degree of relevance is assumed for words that appear frequently in a particular document, but relatively infrequently across a wide range of other documents.

In [ ]:
# install textblob library and define functions for TF-IDF
!pip install -U textblob==0.10.0
import math
from textblob import TextBlob as tb

def tf(word, doc):
    return doc.words.count(word) / len(doc.words)

def contains(word, docs):
    return sum(1 for doc in docs if word in doc.words)

def idf(word, docs):
    return math.log(len(docs) / (1 + contains(word, docs)))

def tfidf(word, doc, docs):
    return tf(word,doc) * idf(word, docs)


# Create a collection of documents as textblobs
doc1 = tb(txt)
doc2 = tb(txt2)
doc3 = tb(txt3)
docs = [doc1, doc2, doc3]

# Use TF-IDF to get the three most important words from each document
print('-----------------------------------------------------------')
for i, doc in enumerate(docs):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, doc, docs) for word in doc.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))


### Stemming

#### View frequency of unstemmed words from Lewis Capaldi's song

In [ ]:
# Load and print the lyrics to the song "Someone You Loved" by Lewis Capaldi
!wget -O SomeoneYouLoved.txt https://raw.githubusercontent.com/markcubanai/Language/master/SomeoneYouLoved.txt
doc4 = open("SomeoneYouLoved.txt", "r")
doc4Txt = doc4.read()

print("------------------------------------------------")
print("Lyrics to 'Someone You Loved' by Lewis Capaldi")
print("------------------------------------------------")
print(doc4Txt)

# Normalize and remove stop words
from string import punctuation
doc4Txt = ''.join(c for c in doc4Txt if not c.isdigit())
doc4Txt = ''.join(c for c in doc4Txt if c not in punctuation).lower()
doc4Txt = ' '.join([word for word in doc4Txt.split() if word not in (stopwords.words('english'))])

# Get Frequency distribution
words = nltk.tokenize.word_tokenize(doc4Txt)
fdist = FreqDist(words)
count_frame = pd.DataFrame(fdist, index =[0]).T
count_frame.columns = ['Count']

# Plot frequency
counts = count_frame.sort_values('Count', ascending = False)
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()    
counts['Count'][:60].plot(kind = 'bar', ax = ax)
ax.set_title('Frequency of the most common words')
ax.set_ylabel('Frequency of word')
ax.set_xlabel('Word')
plt.show()

#### Stem the words using the Porter stemmer

In [ ]:
from nltk.stem.porter import PorterStemmer

# Get the word stems
ps = PorterStemmer()
stems = [ps.stem(word) for word in words]

# Get Frequency distribution
fdist = FreqDist(stems)
count_frame = pd.DataFrame(fdist, index =[0]).T
count_frame.columns = ['Count']

# Plot frequency
counts = count_frame.sort_values('Count', ascending = False)
fig = plt.figure(figsize=(16, 9))
ax = fig.gca()    
counts['Count'][:60].plot(kind = 'bar', ax = ax)
ax.set_title('Frequency of the most common words')
ax.set_ylabel('Frequency of word')
ax.set_xlabel('Word')
plt.show()

### Language Service
#### Create an Azure Cognitive Service for Language

## Using the Language Service
The previous examples demonstrate some ways to write code and analyze text, and they serve to illustrate that text analytics involves applying statistical techniques to text data in order to discern semantic meaning. This is a common theme in many AI solutions.

Microsoft Cognitive Services includes a Language Service that encapsulates much more sophisticated techniques for ascertaining meaning from text.

### Create a Language Service
To see this in action, you need to provision a Language service in your Azure subscription. Follow these steps to do that:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ Create a resource**, and search for "**Language service**".
4. Click **Create**
5. Select **Custom question answering**, then **Continue to create your resource**
5. In the **Create Language** blade, enter the following details, and then click **Review + Create**
  * **Subscription**: Azure for Students or Microsoft Azure Sponsorship
  * **Resource Group**: Choose the existing resource group you created in the previous lab (or create a new one if you didn't complete the previous lab by clicking **Create new** and entering a unique name)
  * **Region**: (US) South Central US
  * **Name**: A unique name for your service.
  * **Pricing tier**: Choose the Free F0 or Standard S pricing tier.
7. When deployment is complete, click **Go to resource**.
8. In the blade for your Language service, click **Keys and Endpoint** and then copy **Key 1** to the clipboard and paste it into the **textKey** variable assignment value in the cell below. 
9. Also copy the **Endpoint** to the clipboard and paste it into the **textAnalyticsEndpoint** variable assignment value in the cell below.
10. Click the **Run** button to run the cell below that assigns the variables textKey and textAnalyticsEndpoint.

In [ ]:
textKey = 'YOUR_KEY_1'
textAnalyticsEndpoint = 'https://YOUR_SERVICE_NAME.cognitiveservices.azure.com/'

#### Analyze the Shark Tank Pitch and Game-Winner Play-By-Play

In [ ]:
try:
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Convert endpoint to URI
    textAnalyticsURI = textAnalyticsEndpoint
    if textAnalyticsURI.startswith('https://'):
      textAnalyticsURI = textAnalyticsURI[len('https://'):]
    while textAnalyticsURI.endswith('/'):
      textAnalyticsURI = textAnalyticsURI[:-1]

    # Define the request headers.
    headers = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': textKey,
        'Accept': 'application/json'
    }

    # Define the parameters
    params = urllib.parse.urlencode({
    })

    # Define the request body
    body = {
        
    "kind": "KeyPhraseExtraction",
    "parameters": {
        "modelVersion": "latest"
    },
    "analysisInput":{
        "documents":[
        {
            "language": "en",
            "id": "1",
            "text": doc1Txt
        },
        {
            "language": "en",
            "id": "2",
            "text": doc3Txt
        }
      ]
    }
    }

    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(textAnalyticsURI)
        conn.request("POST", "/language/:analyze-text?api-version=2022-05-01", str(body), headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")

        # 'data' contains the JSON response, which includes a collection of documents.
        parsed = json.loads(data)
        print(parsed)
        for document in parsed['results']['documents']:
            print("Document " + document["id"] + " key phrases:")
            for phrase in document['keyPhrases']:
                print("  " + phrase)
            print("---------------------------")
        conn.close()

    except Exception as e:
        print('Error:')
        print(e)
except:
    print("Make sure you run the code cell above that assigns the textKey variable! Then run this cell again.")

### Perform Sentiment Analysis
Another common AI requirement is to determine the sentiment associated with some text. For example, you might analyze tweets that include your organization's twitter handle to determine if they are positive or negative.

Run the cell below to use the **sentiment** method of the Text Analytics service to discern the sentiment of two sentences.

In [ ]:
body = {
  "documents": [
    {
      "language": "en",
      "id": "1",
      "text": "Wow! cognitive services are fantastic."
    },
    {
      "language": "en",
      "id": "2",
      "text": "I hate it when computers don't understand me."
    }
  ]
}


try:
    conn = http.client.HTTPSConnection(textAnalyticsURI)
    conn.request("POST", "/text/analytics/v3.1/sentiment?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read().decode("UTF-8")
    parsed = json.loads(data)
    
    # Get the sentiment for each document
    for document in parsed['documents']:
        print("Document:" + document["id"] + " = " + max(document['confidenceScores']))
    conn.close()
    
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



### Create a Speech Service
The Microsoft Cognitive Services include the Speech service, that can interpret spoken input from a microphone or audio file. Follow these steps to provision the Speech service:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ Create a resource**, and search for "**Speech**".
4. In the list of services, click ** Speech**.
5. In the **Speech** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Azure for Students or Microsoft Azure Sponsorship
  * **Location**: (US) South Central US
  * **Pricing tier**: Choose the Free F0 or Standard S0 pricing tier.
  * **Resource Group**: Choose the existing resource group you used previously.
6. Wait for the service to be created.
7. When deployment is complete, click **Go to resource**.
8. In the blade for your Speech service, click **Keys and Endpoint** and then copy **Key 1** to the clipboard and paste it into the **speechKey** variable assignment value in the cell below. 
9. Paste the **Location/Region** value into the **speech_region** variable assignment value in the cell below.
10. Click the Run button to run the cell below that assigns the variable speechKey.

In [ ]:
speechKey = 'YOUR_KEY_1'
speech_region = 'YOUR_REGION'

#### Get the audio input
We'll use a WAV file of captured audio containing the speech we want to transcribe

In [ ]:
import IPython

try:
    speechKey
except:
    print("Make sure you run the code cell above that assigns the speechKey variable! Then run this cell again.")

!wget -O RainSpain.wav https://raw.githubusercontent.com/markcubanai/Language/master/RainSpain.wav
IPython.display.Audio('RainSpain.wav', autoplay=True)

#### Convert speech to text

In [ ]:
import IPython
import requests, json

with open("RainSpain.wav", mode="rb") as audio_file:
        audio_data =  audio_file.read()
        
# The Speech API requires an access token (valid for 10 mins)
apiEndPoint = "https://" + speech_region + ".api.cognitive.microsoft.com/sts/v1.0/issueToken"
apiKey = speechKey
headers = {"Ocp-Apim-Subscription-Key": apiKey}

# Use the API key to request an access token
response = requests.post(apiEndPoint, headers=headers)
accesstoken = str(response.text)

# Now that we have a token, we can set up the request
speechToTextEndPoint = "https://" + speech_region + ".stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1"
headers = {"Content-type": "audio/wav; codec=audio/pcm; samplerate=16000", 
           "Authorization": "Bearer " + accesstoken}
params = {"language":"en-US"}
body = audio_data

# Connect to server, post the request, and get the result
response = requests.post(speechToTextEndPoint,data=body, params=params, headers=headers)
result = str(response.text)
print(json.loads(result)['DisplayText'])

#### Convert text to speech

In [ ]:
import IPython
import requests, json
from xml.etree import ElementTree

# Get the input text
myText = input('What would you like me to say?: \n')
        
# The Speech API requires an access token (valid for 10 mins)
apiEndPoint = "https://" + speech_region + ".api.cognitive.microsoft.com/sts/v1.0/issueToken"
apiKey = speechKey
headers = {"Ocp-Apim-Subscription-Key": apiKey}

# Use the API key to request an access token
response = requests.post(apiEndPoint, headers=headers)
accesstoken = str(response.text)

# Now that we have a token, we can set up the request
textToSpeechEndPoint = "https://" + speech_region + ".tts.speech.microsoft.com/cognitiveservices/v1"
headers = {"Content-type": "application/ssml+xml",
           'X-Microsoft-OutputFormat': 'riff-16khz-16bit-mono-pcm',
           'User-Agent': 'speech',
           "Authorization": "Bearer " + accesstoken}

# The request body is XML
body = "<speak version='1.0' xml:lang='en-US'>\
          <voice xml:lang='en-US'\
                 xml:gender='Female'\
                 name='en-US-JennyNeural'>" + myText + "</voice>\
        </speak>"


# Connect to server, post the request, and get the result
response = requests.post(textToSpeechEndPoint,data=body, headers=headers)

#Play the audio
IPython.display.Audio(response.content, autoplay=True)

### Translation

### Create a Translator Service
The Microsoft Cognitive Services include the Translator service, that can translate text between different languages. Follow these steps to provision the Translator service:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ Create a resource**, and search for "**Translator**".
4. In the list of services, click **Translator**.
5. In the **Translator** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Review + Create**
  * **Subscription**: Azure for Students or Microsoft Azure Sponsorship
  * **Resource Group**: Choose the existing resource group you used previously.
  * **Region**: Global
  * **Name**: A unique name for your service.
  * **Pricing tier**: Choose the Free F0 or Standard S1 pricing tier.
7. Click **Review + Create**, then click **Create**
8. Wait for the service to be created.
9. When deployment is complete, click **Go to resource**.
10. In the blade for your Translator Text service, click **Keys and Endpoint** and then copy **Key 1** to the clipboard and paste it into the **transTextKey** variable assignment value in the cell below. 
11. Click the **Run** button to run the cell below that assigns the variable transTextKey.

#### Get the service key

In [ ]:
transTextKey = "YOUR_KEY_1"

#### Translate Text
When asked to enter a language, note that you must enter the language code (i.e. "en" for English). Find all compatible language codes here: https://docs.microsoft.com/en-us/azure/cognitive-services/translator/language-support

Sample Language Codes:

 * **English:** en
 * **Spanish:** es
 * **French:** fr
 * **German:** de
 * **Korean:** ko



In [ ]:
try:
    import requests, http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib
    from xml.etree import ElementTree


    textToTranslate = input('Please enter some text: \n')
    fromLangCode = input('What language is this? (Enter language code): \n') 
    toLangCode = input('To what language would you like it translated? (Enter language code): \n') 

    try:
        # Define the request headers.
        headers = {
            'Ocp-Apim-Subscription-Key': transTextKey,
            'Content-Type': 'application/json'
        }

        # Define the parameters
        params = urllib.parse.urlencode({
            "api-version": "3.0",
            "to": toLangCode,
            "from": fromLangCode
        })

        # Define the request body
        requestBody = json.dumps([{"Text": textToTranslate}])

        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection("api.cognitive.microsofttranslator.com")
        conn.request("POST", "/translate?%s" % params, requestBody, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        parsed = json.loads(data)
        print (parsed[0]['translations'][0]['text'])

        conn.close()

    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
except:
    print("Make sure you run the code cell above that assigns the transTextKey variable! Then run this cell again.")

# Nice work!
In this lab, you saw how to use many different Azure tools to process text and speech. There are so many applications you can create by combining these different tools. Brainstorm a few ideas you could build for the final project!